In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
DATA_PATH = os.path.join(PROJ_DIR, "data")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [3]:
ROBOT_TYPE_NAME="indy7"
DATASET_TRAIN = ['20210209-193147', '20210209-213935', '20210209-224826', '20210210-000355']
DATASET_TEST = ['20210210-011407']
ARM_FOLDER = ARM10_FOLDER
GRASP_SHAPE = (20,20,20)
ARM_SHAPE = (20,20,20)
RH_MASK_SIZE = 512
RH_MASK_STEP = 64

ROBOT_DATA_ROOT = os.path.join(DATA_PATH, ROBOT_TYPE_NAME)

## Load & test

In [4]:
last_model = sorted(os.listdir(LAT_MODEL_PATH))[-1]

In [5]:
logpath = os.path.join(LAT_MODEL_PATH, last_model)
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
last_save = "model_5"
model_log_dir = os.path.join(logpath, last_save)

In [6]:
import tensorflow as tf
model = tf.keras.models.load_model(model_log_dir)

In [7]:
@tf.function
def inference(images):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    return predictions

## prepare shared array server

In [8]:
import SharedArray as sa
BATCH_SIZE = 1
SERVER_PERIOD = 1e-3

In [9]:
from pkg.controller.robot_config import RobotType
int2rtypename = {v.value:v.name for v in RobotType}

# Create an array in shared memory.
robot_type_p = sa.create("shm://robot_type", (1,), dtype=np.int)
grasp_img_p = sa.create("shm://grasp_img", (BATCH_SIZE,)+GRASP_SHAPE+(2,))
arm_img_p = sa.create("shm://arm_img", (BATCH_SIZE,)+ARM_SHAPE+(1,))
rh_mask_p = sa.create("shm://rh_mask", (BATCH_SIZE, RH_MASK_SIZE*2))
result_p = sa.create("shm://result", (BATCH_SIZE, 2))
query_in = sa.create("shm://query_in", (1,), dtype=np.bool)
response_out = sa.create("shm://response_out", (1,), dtype=np.bool)
query_quit = sa.create("shm://query_quit", (1,), dtype=np.bool)
grasp_img_p[:]=0
arm_img_p[:]=0
rh_mask_p[:]=0
result_p[:]=0
query_in[0] = False
response_out[0] = False
query_quit[0] = False

print("============= wait for initialization ================")
inference([grasp_img_p, arm_img_p, rh_mask_p])
print("=============== initialization done ==================")


try:
    while not query_quit[0]:
        if not query_in[0]:
            time.sleep(SERVER_PERIOD)
            continue
        query_in[0] = False
        rtypename = int2rtypename[robot_type_p[0]]
        ## TODO: inference depending on robot type
        result = inference([grasp_img_p, arm_img_p, rh_mask_p])
        for i_b in range(BATCH_SIZE):
            result_p[i_b] = result[i_b]
        response_out[0] = True
finally:
    sa.delete("shm://robot_type")
    sa.delete("shm://grasp_img")
    sa.delete("shm://arm_img")
    sa.delete("shm://rh_mask")
    sa.delete("shm://result")
    sa.delete("shm://query_in")
    sa.delete("shm://response_out")
    sa.delete("shm://query_quit")

============= wait for initialization ================
=============== initialization done ==================


KeyboardInterrupt: 